In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [2]:
# Import Data
demo_data = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')
demo_data.head()

,age,income,days_as_member,gender_F,gender_M,offer_viewed,offer_completed,viewed_before_completion,difficulty,reward,duration_hrs,mobile,social,web,is_bogo,is_discount,total_transactions,total_transaction_amount
customer_id,,,,,,,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0,1,0,5,5,168,1,0,1,1,0,1,23.22
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0,0,0,0,0,96,1,0,1,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,1,1,5,5,168,1,0,1,1,0,1,19.89
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,0,0,0,0,72,1,1,0,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,1,1,10,10,168,1,1,0,1,0,1,21.72


# Assess Features

In [3]:
# Group independent variables
all_ind_vars = (
    ['age','income', 'days_as_member'] #demographic data
    + ['is_bogo', 'is_discount']       # offer_type
    + ['difficulty', 'reward', 'duration_hrs'] # offer description
    + ['mobile', 'social', 'web'] # offer distribution channel
)

# Target var
offer_completed = 'offer_completed'

# Since we want to uncover demographic response trends, we need to split the data by gender
# and filter on the cases where the customer viewed the offer
female_df = demo_data[(demo_data['gender_F'] == 1) & (demo_data['offer_viewed'] == 1)]
male_df = demo_data[(demo_data['gender_M'] == 1) & (demo_data['offer_viewed'] == 1)]


# Analyze Feature Correlations

In [4]:
def analyze_feature_correlation(df, independent_vars, target_var):
    # Define independent vars and target var
    X = df[independent_vars]
    y = df[target_var]

    # Combine X and y for correlation calculation
    combined_df = pd.concat([X, y], axis=1)

    # Calculate correlations
    correlations = combined_df.corr()[target_var].drop(target_var)

    # Create a DataFrame for easier visualization
    features_correlation = pd.DataFrame({'feature': correlations.index, 'correlation': correlations.values})

    # Sort the DataFrame by correlation
    features_correlation_sorted = features_correlation.sort_values(by='correlation', ascending=False).reset_index(drop=True)

    return features_correlation_sorted

# Call the function
female_corr = analyze_feature_correlation(female_df, independent_vars=all_ind_vars, target_var=offer_completed)
male_corr = analyze_feature_correlation(male_df, independent_vars=all_ind_vars, target_var=offer_completed)

# Analyze Feature Importance

In [5]:
def analyze_feature_importance(df, independent_vars, target_var):
    # Define independent vars
    X = df[independent_vars]
    
    # Define target var
    y = df[target_var]  

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Decision Tree Classifier
    clf = DecisionTreeClassifier(random_state=42)

    # Fit the model
    clf.fit(X_train, y_train)

    # Get feature importances
    feature_importances = clf.feature_importances_

    # Create a DataFrame for easier visualization
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

    # Sort the DataFrame by importance
    features_sorted = features.sort_values(by='importance', ascending=False).reset_index(drop=True)

    return features_sorted


# Call the function
female_import = analyze_feature_importance(female_df, independent_vars=all_ind_vars, target_var=offer_completed)
male_import = analyze_feature_importance(male_df, independent_vars=all_ind_vars, target_var=offer_completed)

In [6]:
# Join the data together
women = pd.merge(female_corr, female_import, on='feature', how='inner')

# Calculate the magnitude of each feature
women['magnitude'] = abs(women['correlation'] + women['importance'])

# Round the values and sort the data frame
women = women.round(2)
women = women.sort_values(by='magnitude', ascending=False).reset_index(drop=True)

# women.to_csv(r'data/04_fct/fct_feature_analysis_women.csv')
women.head()

,feature,correlation,importance,magnitude
0,reward,0.29,0.42,0.70
1,duration_hrs,0.47,0.02,0.48
2,difficulty,0.44,0.01,0.45
3,days_as_member,0.17,0.25,0.41
4,is_discount,0.31,0.00,0.31


In [7]:
# Join the data together
men_df = pd.merge(male_corr, male_import, on='feature', how='inner')

# Calculate the magnitude of each feature
men_df['magnitude'] = abs(men_df['correlation'] + men_df['importance'])

# Round the values and sort the data frame
men_df = men_df.round(2)
men_df = men_df.sort_values(by='magnitude', ascending=False).reset_index(drop=True)

# men_df.to_csv(r'data/04_fct/fct_feature_analysis_men.csv')
men_df.head()

,feature,correlation,importance,magnitude
0,days_as_member,0.17,0.32,0.50
1,duration_hrs,0.42,0.02,0.44
2,reward,0.12,0.29,0.40
3,difficulty,0.35,0.01,0.36
4,income,0.19,0.16,0.35
